# Import packages:

In [2]:
from openbiolink.obl2021 import OBL2021Dataset
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import torch
import math

D:\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Accessing Data:

In [3]:
obl_dataset = OBL2021Dataset()

Dataset found in D:\programming repo-1\university of glasgow\msc project\MSc.Project\obl2021, omitting download...


In [4]:
# get the training, validation and test sets

#obl_train_dataset = obl_dataset.training
#obl_valid_dataset = obl_dataset.validation
#obl_test_dataset = obl_dataset.testing

In [6]:
#checking out the tensor shape
print('The shape of the training tensor is : ', obl_dataset.training.shape)

#checking out the tensor dtype
print('The the data type of the training tensor is : ', obl_dataset.training.dtype)

The shape of the training tensor is :  torch.Size([4192002, 3])
The the data type of the training tensor is :  torch.int64


# Embedding Models:

## TransE:

In [7]:
class TransE(nn.Module):
    def __init__(self, device, num_entity, num_relation, emb_dim, gamma):
        super(TransE, self).__init__()
        self.device = device
        self.emb_dim = emb_dim
        self.num_entity = num_entity
        self.num_relation = num_relation
        
        #initialize entity embeddings
        self.entity_emb = self.initialize_emb(self.num_entity, self.emb_dim)

        #initialie relation embeddings
        self.relation_emb = self.initialize_emb(self.num_relation, self.emb_dim)
        self.relation_emb.weight.data.div_(self.relation_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        #create the loss function
        loss_fn = nn.MarginRankingLoss(margin=gamma)
        

    def initialize_emb(self, num_emb, emb_dim):
        emb_weight_range = 6 / math.sqrt(emb_dim)
        emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        emb.weight.data.uniform_( -emb_weight_range, emb_weight_range )
        return emb
    
    def forward(self, pos_triplet, neg_triplet):
        self.entity_emb.weight.data.div_(self.entity_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        pos_distance = cal_distance(pos_triplet)
        neg_distance = cal_distance(neg_triplet)
        return loss_fn(pos_distance, neg_distance, torch.tensor([-1], dtype=torch.int64, device=self.device))
    
    def cal_distance(self, triplet):
        head = triplet[:,0]
        relation = triplet[:,1]
        tail = triplet[:,2]
        return (self.entity_emb(head) + self.relation(relation) - self.entity_emb(tail))
       
        

In [8]:
#test transe_model, remove later
transe_model = TransE(device = 'cuda', num_entity = 100, num_relation = 10, emb_dim = 100, gamma = 0.1)

### Create Dataset:

In [9]:
obl_train_dataset = TensorDataset(obl_dataset.training)
obl_valid_dataset = TensorDataset(obl_dataset.validation)
obl_test_dataset = TensorDataset(obl_dataset.testing)

### Create DataLoader:

In [10]:
def create_dataloader(dataset, batch_size, shuffle):
    tensor_dataset = TensorDataset(dataset)
    return DataLoader(tensor_dataset, batch_size=batch_size, shuffle=shuffle) 

### Training Function:

In [11]:
def train_transe(data_loader, epoch, device):
    for i in range(epoch):
        for index, batch_data in enumerate(data_loader):  
            print(len(batch_data))

### Testing Function:

In [12]:
def test_transe():
    pass

### Set parameters and call train TransE:

In [13]:
transe_model = TransE(device = 'cuda', num_entity = 100, num_relation = 10, emb_dim = 100, gamma = 0.1)
optimizer = torch.optim.Adam(transe_model.parameters(), lr=0.0001)

data_loader = create_dataloader(obl_train_dataset, batch_size=128, shuffle=True)
train_transe(data_loader=data_loader, epoch=20, device='cuda')

AttributeError: 'TensorDataset' object has no attribute 'size'